In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from lxml import etree
import re

import time

import requests
from PyPDF2 import PdfReader
from io import BytesIO

import chardet

#from threading import Thread

In [15]:
# 全局变量
sleep = 0.3

# 加载上市公司主页信息
with open('index2.txt', 'r+') as index:
	index_url_list:list = index.read().split('\n')

# 加载关键词组
with open('keywords.txt', 'r+', encoding='utf-8') as keywords:
	keywords_list:list = keywords.read().split('\n')
pattern = "|".join(keywords_list)

# 已经记录过的上市公司
with open('getted.txt', 'a+') as getted:
	getted.seek(0)
	getted_list = getted.read().split('\n')

In [16]:
driver = webdriver.Chrome()

In [17]:
# 从上市公司主页#periodicReports找年报连接
for url in index_url_list:
	# 提取上市公司代码
	# print(url)
	try:
		stkid = re.search('stockCode=(\d{6})', url).group(1)
		if stkid not in getted_list:
			status = '404'
			while status == '404':
				driver.get(url)
				status = driver.execute_script("return window.status")
			'''
			由于只要20、21年的，所以这里没有做翻页		
			'''
			reports_list = driver.find_elements(By.XPATH, '//*[@id="main"]/div[3]/div/div[2]/div/div/div[2]/div[1]/div[2]/div/div/div[3]/table/tbody/tr')
			for row in reports_list: 
				row_html = etree.fromstring(row.get_attribute('outerHTML'))
				title = row_html.xpath('./td/div/a/span[1]')[0].text

				if (not re.search(r'(摘要|正文|补充公告|更正前|已取消)', title)) & (re.search('(2020|2021)年年度报告', title) != None):
					year = re.match(r'.+(\d{4}).+', title).group(1)
					link = 'http://www.cninfo.com.cn' + row_html.xpath('./td/div/a')[0].get('href')
					# 打开新的标签页来取得pdf连接
					script = "window.open('" + link + "','_blank');"
					# print(script)
					driver.execute_script(script)
					driver.switch_to.window(driver.window_handles[-1])
					while re.search('.+\.PDF', link) == None:
						link = driver.find_element(By.XPATH, '//*[@id="noticeDetail"]/div/div[2]/div[1]/a').get_attribute('href')
						time.sleep(sleep)
					# 解析pdf
					pdf_content = requests.get(link).content
					pdf_file = BytesIO(pdf_content)
					pdf_reader = PdfReader(pdf_file)
					count = 0
					for page in pdf_reader.pages:
						content = page.extract_text()
						count += len(re.findall(pattern, content))
					temp = [stkid, year, str(count), link]
					with open('pdf_url.txt', 'a+') as pdf_url:
						pdf_url.write('\t'.join(temp)+'\n')
					# 关闭此标签页并返回
					driver.close()
					driver.switch_to.window(driver.window_handles[0])
			# 结束
			getted_list.append(stkid)
			with open('getted.txt', 'a+') as getted:
				getted.write(stkid + '\n')
	except AttributeError:
		print(url)

http://www.cninfo.com.cn/new/disclosure/stock?stockCode=undefined&orgId=undefined#periodicReports


e:\anaconda3\lib\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


http://www.cninfo.com.cn/new/disclosure/stock?stockCode=undefined&orgId=undefined#periodicReports


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x00706643]
	(No symbol) [0x0069BE21]
	(No symbol) [0x0059DA9D]
	(No symbol) [0x0057EF6A]
	(No symbol) [0x005F3AAB]
	(No symbol) [0x006061B6]
	(No symbol) [0x005EFB76]
	(No symbol) [0x005C49C1]
	(No symbol) [0x005C5E5D]
	GetHandleVerifier [0x0097A142+2497106]
	GetHandleVerifier [0x009A85D3+2686691]
	GetHandleVerifier [0x009ABB9C+2700460]
	GetHandleVerifier [0x007B3B10+635936]
	(No symbol) [0x006A4A1F]
	(No symbol) [0x006AA418]
	(No symbol) [0x006AA505]
	(No symbol) [0x006B508B]
	BaseThreadInitThunk [0x76CF7D69+25]
	RtlInitializeExceptionChain [0x779FBB9B+107]
	RtlClearBits [0x779FBB1F+191]


In [36]:
url = r'深康佳Ａ：2022年三季度报告'

a = re.search('(2020|2021)年年度报告', url)

a == None

True